<a href="https://colab.research.google.com/github/luntrum/model_predict_age_gender_and_faceshape/blob/main/faceshape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
import math as m
import math
import keras_preprocessing
import seaborn as sns
import pandas as pd

from PIL import Image, ImageOps

from keras import optimizers
from keras.models import Model, Sequential,load_model
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation,Conv2D
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.models import model_from_json
from tensorflow.keras.optimizers import Adam 


from sklearn.model_selection import train_test_split


from os import listdir

In [ ]:
im_shape = (200,200)

TRAINING_DIR = '/content/gdrive/MyDrive/Colab Notebooks/cuoiki/FaceShape Dataset/testing_set'
TEST_DIR = '/content/gdrive/MyDrive/Colab Notebooks/cuoiki/FaceShape Dataset/training_set'

seed = 10

BATCH_SIZE = 16
data_generator = ImageDataGenerator(
        validation_split=0.2,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

val_data_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2)
# Generator data train
train_generator = data_generator.flow_from_directory(TRAINING_DIR, target_size=im_shape, shuffle=True, seed=seed,
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="training")
# Generator data validation
validation_generator = val_data_generator.flow_from_directory(TRAINING_DIR, target_size=im_shape, shuffle=False, seed=seed,
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="validation")

# Generator data test
test_generator = ImageDataGenerator(rescale=1./255)
test_generator = test_generator.flow_from_directory(TEST_DIR, target_size=im_shape, shuffle=False, seed=seed,
                                                     class_mode='categorical', batch_size=BATCH_SIZE)

nb_train_samples = train_generator.samples
nb_validation_samples = validation_generator.samples
nb_test_samples = test_generator.samples
classes = list(train_generator.class_indices.keys())
print('Classes: '+str(classes))
num_classes  = len(classes)

In [ ]:
agemodel = Sequential()
agemodel.add(Conv2D(32, (3,3), activation='relu', input_shape=(200, 200, 3)))
agemodel.add(MaxPooling2D((2,2)))
agemodel.add(Conv2D(64, (3,3), activation='relu'))
agemodel.add(MaxPooling2D((2,2)))
agemodel.add(Conv2D(128, (3,3), activation='relu'))
agemodel.add(MaxPooling2D((2,2)))
agemodel.add(Flatten())
agemodel.add(Dense(64, activation='relu'))
agemodel.add(Dropout(0.5))
agemodel.add(Dense(1, activation='relu'))

agemodel.compile(loss='mean_squared_error',
             optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
             metrics=['accuracy'])

genmodel = Sequential()
genmodel.add(Conv2D(32, (3,3), activation='relu', input_shape=(200, 200, 3)))
genmodel.add(MaxPooling2D((2,2)))
genmodel.add(Conv2D(64, (3,3), activation='relu'))
genmodel.add(MaxPooling2D((2,2)))
genmodel.add(Conv2D(128, (3,3), activation='relu'))
genmodel.add(MaxPooling2D((2,2)))
genmodel.add(Flatten())
genmodel.add(Dense(64, activation='relu'))
genmodel.add(Dropout(0.5))
genmodel.add(Dense(1, activation='sigmoid'))

genmodel.compile(loss='binary_crossentropy',
             optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
             metrics=['accuracy'])



faceshapemodel = Sequential()
faceshapemodel.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(im_shape[0],im_shape[1],3)))
faceshapemodel.add(MaxPooling2D(pool_size=(2, 2)))
faceshapemodel.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
faceshapemodel.add(Flatten())
faceshapemodel.add(Dense(128, activation='relu'))
faceshapemodel.add(Dropout(0.2))
faceshapemodel.add(Dense(num_classes, activation='softmax'))
faceshapemodel.summary()

# Compile model
faceshapemodel.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

Model: "sequential_62"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_176 (Conv2D)         (None, 198, 198, 32)      896       
                                                                 
 max_pooling2d_173 (MaxPooli  (None, 99, 99, 32)       0         
 ng2D)                                                           
                                                                 
 conv2d_177 (Conv2D)         (None, 97, 97, 64)        18496     
                                                                 
 flatten_43 (Flatten)        (None, 602176)            0         
                                                                 
 dense_123 (Dense)           (None, 128)               77078656  
                                                                 
 dropout_62 (Dropout)        (None, 128)               0         
                                                     

In [ ]:
history3 = faceshapemodel.fit(train_generator,
        steps_per_epoch=nb_train_samples // 16,
        epochs=50,
        
        validation_data=validation_generator,
        verbose = 1,
        validation_steps=nb_validation_samples // 16)

Epoch 1/50
50/50 [==============================] - 358s 7s/step - loss: 4.9944 - accuracy: 0.2188 - val_loss: 1.6093 - val_accuracy: 0.2031
Epoch 2/50
50/50 [==============================] - 58s 1s/step - loss: 1.6094 - accuracy: 0.2163 - val_loss: 1.6095 - val_accuracy: 0.1875
Epoch 3/50
50/50 [==============================] - 56s 1s/step - loss: 1.6102 - accuracy: 0.2375 - val_loss: 1.6072 - val_accuracy: 0.2083
Epoch 4/50
50/50 [==============================] - 56s 1s/step - loss: 1.6088 - accuracy: 0.2225 - val_loss: 1.6101 - val_accuracy: 0.1771
Epoch 5/50
50/50 [==============================] - 56s 1s/step - loss: 1.6092 - accuracy: 0.1963 - val_loss: 1.6087 - val_accuracy: 0.2031
Epoch 6/50
50/50 [==============================] - 56s 1s/step - loss: 1.6070 - accuracy: 0.2212 - val_loss: 1.6069 - val_accuracy: 0.2031
Epoch 7/50
50/50 [==============================] - 57s 1s/step - loss: 1.6115 - accuracy: 0.2362 - val_loss: 1.6091 - val_accuracy: 0.2083
Epoch 8/50
50/50 [=

In [ ]:
faceshapemodel.save('/content/gdrive/MyDrive/Colab Notebooks/cuoiki/faceshape.h')

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/Colab Notebooks/cuoiki/faceshape.h/assets
